## Exercise 1:
In class we have gone through a rule-based NLP pipeline by executing components one by one. Typically, we create a pipeline as a single class that links all modules.

In this exercise, you will need to write a pipeline class. Let's call it **MyPipe**, it can be initiated through taking a set of rules. After that, it can be called through a **process** function, which can take a document text, and output a set of values: 
- document level classification
- context document, which is an object of type pyConTextGraph.ConTextDocument;
- annotations (in dataframe format);
- relations (in dataframe format).

In [1]:
# state your import here

from PyRuSH.RuSH import RuSH
from pyConTextNLP import pyConTextGraph
from pyConTextNLP.utils import get_document_markups


from DocumentClassifier import FeatureInferencer
from DocumentClassifier import DocumentInferencer
from nlp_pneumonia_utils import markup_sentence
from itemData import get_item_data
from visual import convertMarkups2DF


# begin to define MyPipe class
class MyPipe:
    def __init__(self, sentence_rules, target_rules, context_rules, feature_inference_rule, document_inference_rule):
        # initiate necessary components here
        self.sentence_rules = sentence_rules
        self.target_rules = target_rules
        self.context_rules = context_rules
        self.feature_inference_rule = feature_inference_rule
        self.document_inference_rule = document_inference_rule
        
        pass
    
    def process(self, doc_text):        
        #process your input doc_text, return the required results
        self.doc_text = doc_text
        
        
        
        return doc_class, context_doc, annotations, relations

Once your pipeline class is defined, you can use it multiple times for different set of rules.

## Exercise 2:
Now you can select documents from the MIMIC database (limit to 5 documents that contain your target concept), write a script to process all of them, and output a dictionary which uses document name as keys and document level classification as values.

In [2]:
# configure your rules 
sentence_rules='KB/rush_rules.tsv'
target_rules='''
Comments: ''
Direction: ''
Lex: fever
Regex: ''
Type: FEVER
---
Comments: ''
Direction: ''
Lex: high temperature
Regex: '1\d\d\.\d F'
Type: FEVER'''
context_rules='''
Comments: ''
Direction: forward
Lex: 'no'
Regex: ''
Type: DEFINITE_NEGATED_EXISTENCE
---
Comments: ''
Direction: forward
Lex: 'denies'
Regex: ''
Type: DEFINITE_NEGATED_EXISTENCE
'''
feature_inference_rule='''
#Conclusion type, Evidence type, Modifier values associated with the evidence
NEGATED_CONCEPT,FEVER,DEFINITE_NEGATED_EXISTENCE
'''
document_inference_rule='''
#Conclusion Type at document level, Evidence type at mention level
FEVER_DOC,FEVER

#Default document type
NO_FEVER
'''


In [3]:
#initiate an instance of MyPipe
myPipe=MyPipe(sentence_rules, target_rules, context_rules, feature_inference_rule, document_inference_rule)

In [4]:
# write you code here, fill the results by processing each document through MyPipe
results=dict()  # this dictionary will contain document names as keys and a document-level classification as values.



In [5]:
print(results)

{}


## Exercise 3:

Now you get the results, but how can you be sure if they are correct? Let's dig a little deeper to visualize them. 

Hint: **view_pycontext_output** can take in a list of ConTextDocuments and visualize them.


In [6]:
from visual import view_pycontext_output
from visual import Vis

In [7]:
#get the context documents that you created in Exercise 2.
context_docs=[]



Change the file name to contain your UNID.

In [8]:
view_pycontext_output(context_docs, Vis(file_name="u1166466.html"))

AttributeError: 'list' object has no attribute 'getDocument'

***Please make a screenshot of the visualization file and submit it as homework assignment.***